<a href="https://colab.research.google.com/github/marreapato/Deep_Learning_Course/blob/main/TF_IDF_pyspartfidf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/mlcrs/

!ls

/content/gdrive/MyDrive/mlcrs
access_log.txt			      KFoldCrossValidation.ipynb     Percentiles.ipynb
breakfast.jpg			      KMeans.ipynb		     PoliticsExercise.ipynb
bridge.jpg			      LinearRegression.ipynb	     PolynomialRegression.ipynb
bunny.jpg			      mammographic_masses.data.txt   Python101.ipynb
castle.jpg			      mammographic_masses.names.txt  regression.txt
ConditionalProbabilityExercise.ipynb  mammo_masses_project.ipynb     Seaborn.ipynb
ConditionalProbabilitySolution.ipynb  MatPlotLib.ipynb		     SparkDecisionTree.py
CovarianceCorrelation.ipynb	      MeanMedianExercise.ipynb	     SparkKMeans.py
DecisionTree.ipynb		      MeanMedianMode.ipynb	     SparkLinearRegression.py
DeepLearningProject.ipynb	      ml-100k			     SparkPCA.py
DeepLearningProject-Solution.ipynb    MLCourse.zip		     StdDevVariance.ipynb
distancerecomendation-KNN.ipynb       mlcrsItemBasedCF.ipynb	     subset-small.tsv
Distributions.ipynb		      mlcrsQ-Learning.ipynb	     SVC.ipynb
emails				      mlcrs_Sim

In [4]:
data_dir = "/content/gdrive/MyDrive/mlcrs/subset-small.tsv"

In [5]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [662 kB]
Get:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,600 kB]
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 https://ppa.laun

In [7]:
from pyspark import SparkConf, SparkContext
from pyspark.mllib.feature import HashingTF
from pyspark.mllib.feature import IDF

# Boilerplate Spark stuff:
conf = SparkConf().setMaster("local").setAppName("SparkTFIDF")
sc = SparkContext.getOrCreate(conf = conf)

# Load documents (one per line).
rawData = sc.textFile(data_dir)
fields = rawData.map(lambda x: x.split("\t"))
documents = fields.map(lambda x: x[3].split(" "))



In [8]:
rawData

/content/gdrive/MyDrive/mlcrs/subset-small.tsv MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0

In [9]:
# Store the document names for later:
documentNames = fields.map(lambda x: x[1])

# Now hash the words in each document to their term frequencies:
hashingTF = HashingTF(100000)  #100K hash buckets just to save some memory
tf = hashingTF.transform(documents)



In [10]:
tf

PythonRDD[2] at RDD at PythonRDD.scala:53

In [11]:
# At this point we have an RDD of sparse vectors representing each document,
# where each value maps to the term frequency of each unique hash value.

# Let's compute the TF*IDF of each term in each document:
tf.cache()
idf = IDF(minDocFreq=2).fit(tf)
tfidf = idf.transform(tf)

# Now we have an RDD of sparse vectors, where each value is the TFxIDF
# of each unique hash value for each document.

# I happen to know that the article for "Abraham Lincoln" is in our data
# set, so let's search for "Gettysburg" (Lincoln gave a famous speech there):

# First, let's figure out what hash value "Gettysburg" maps to by finding the
# index a sparse vector from HashingTF gives us back:
gettysburgTF = hashingTF.transform(["Gettysburg"])
gettysburgHashValue = int(gettysburgTF.indices[0])

# Now we will extract the TF*IDF score for Gettsyburg's hash value into
# a new RDD for each document:
gettysburgRelevance = tfidf.map(lambda x: x[gettysburgHashValue])

# We'll zip in the document names so we can see which is which:
zippedResults = gettysburgRelevance.zip(documentNames)

# And, print the document with the maximum TF*IDF value:
print("Best document for Gettysburg is:")
print(zippedResults.max())

Best document for Gettysburg is:
(0.0, 'Ælle of Sussex')
